In [25]:
import pandas as pd
import numpy as np
from pandasql import sqldf
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import calendar

In [26]:
today = datetime.today()

#generate m-1

if today.month == 1:
    previous_month = 12
    previous_year = today.year -1
else:
    previous_month = today.month - 1
    previous_year = today.year
    
filename = f"./input/product_{previous_year}-{previous_month:02d}.csv"

data_retail = pd.read_csv(filename)

#data_retail.head()

In [53]:
data_retail_cleanup = sqldf('''select * from data_retail
                            where product_id != 'product_id' and product_id is not null
                            and replace(description, ',', ' ')''')

data_retail_cleanup.tail()

,product_id,invoice_no,stock_code,description,quantity,invoice_date,unit_price,cumtomer_id,product_group,product_name,stock
61,CK227,564184,84991,60 TEATIME FAIRY CAKE CASES,24,8/23/2024 15:11,0.55,NaN,KITCHENWARE,CAKE CASE,1132.0
62,CK227,564565,84991,60 TEATIME FAIRY CAKE CASES,24,8/26/2024 10:50,0.55,14911.0,KITCHENWARE,CAKE CASE,1108.0
63,CK227,562670,84992,72 SWEETHEART FAIRY CAKE CASES,24,8/8/2024 13:42,0.55,14911.0,KITCHENWARE,CAKE CASE,2536.0
64,CK227,564184,84992,72 SWEETHEART FAIRY CAKE CASES,48,8/23/2024 15:11,0.55,NaN,KITCHENWARE,CAKE CASE,2488.0
65,CK227,564565,84992,72 SWEETHEART FAIRY CAKE CASES,48,8/26/2024 10:50,0.55,14911.0,KITCHENWARE,CAKE CASE,2440.0


In [28]:
#read data of customer
customer_rd = pd.read_csv('./input/customer_info.csv', skip_blank_lines = True)
# Hapus kolom 'Unnamed' jika ada
customer_rd = customer_rd.loc[:, ~customer_rd.columns.str.contains('^Unnamed')]

customer_rd.head()

,customer_id,customer_name,email_address,country,phone
0,12386,Allen Bowers,allen_bowers@example.com,Australia,+64 00 34
1,12388,Allison Huang,allison_huang@example.com,Australia,+64 00 35
2,12393,Alyssa Ford,alyssa_ford@example.com,Australia,+64 00 38
3,12415,Amy Campbell,amy_campbell@example.com,Australia,+64 00 56
4,12422,Amy Watson,amy_watson@example.com,Australia,+64 00 61


In [29]:
join_transaction_customer = sqldf('''select 
                                  dr.cumtomer_id as cumtomer_id,
                                  c.customer_name,
                                  c.email_address as email,
                                  c.country, c.phone,
                                  dr.product_id,
                                  dr.invoice_no,
                                  dr.stock_code,
                                  dr.description,
                                  dr.quantity,
                                  dr.unit_price as price,
                                  dr.product_group,
                                  dr.product_name,
                                  dr.stock 
                                  from data_retail_cleanup dr
                                  join customer_rd c on dr.cumtomer_id = c.customer_id
                                  where dr.quantity != 0''')

join_transaction_customer.head()

,cumtomer_id,customer_name,email,country,phone,product_id,invoice_no,stock_code,description,quantity,price,product_group,product_name,stock
0,12621.0,Christina Lopez,christina_lopez@example.com,Germany,+49 00 227,GBG207,564972,23437,50'S CHRISTMAS GIFT BAG LARGE,12,1.25,BAG,GIFT BAG,1590.0
1,12562.0,Bruce Martin,bruce_martin@example.com,France,+33 00 175,PEN209,562450,22418,10 COLOUR SPACEBOY PEN,24,0.85,STATIONERY,PEN,2328.0
2,12473.0,Anthony Sims,anthony_sims@example.com,Germany,+49 00 103,PEN209,563540,22418,10 COLOUR SPACEBOY PEN,24,0.85,STATIONERY,PEN,2304.0
3,12621.0,Christina Lopez,christina_lopez@example.com,Germany,+49 00 227,PEN209,564140,22418,10 COLOUR SPACEBOY PEN,24,0.85,STATIONERY,PEN,2280.0
4,12714.0,Danielle Morris,danielle_morris@example.com,France,+33 00 303,PEN209,564372,22418,10 COLOUR SPACEBOY PEN,24,0.85,STATIONERY,PEN,2256.0


In [51]:
def product_table(data_retail_cleanup):
    # Sort data by product_id
    sort_data_retail_cleanup = data_retail_cleanup.sort_values(by=['product_id'], ascending=True)
    
    # Remove duplicates based on product_id
    dedup_data_retail_cleanup = sort_data_retail_cleanup.drop_duplicates(subset='product_id', keep='first').copy()  # Tambahkan .copy() di sini
    
    # Convert product_group to lowercase
    dedup_data_retail_cleanup.loc[:, 'product_group'] = dedup_data_retail_cleanup['product_group'].str.lower()  # Gunakan .loc[] di sini
    
    # Define location based on product_group
    location_map = {
        'accessories': 'Lt.2_Stand.1_accessories',
        'bag': 'Lt.2_Stand.4_bag',
        'box': 'Lt.1_Stand.1_box',
        'cloth': 'Lt.2_Stand.5_cloth',
        'electronic': 'Lt.1_Stand.2_electronic',
        'glassware': 'Lt.1_Stand.3_glassware',
        'kitchenware': 'Lt.1_Stand.4_kitchenware',
        'personal hygene': 'Lt.2_Stand.6_personal hygene',
        'stationery': 'Lt.2_Stand.2_stationery',
        'utensil': 'Lt.1_Stand.7_utensil',
        'tools': 'Lt.1_Stand.6_tools',
        'toy': 'Lt.2_Stand.3_toy'
    }
    
    # Map product_group ke location
    dedup_data_retail_cleanup.loc[:, 'location'] = dedup_data_retail_cleanup['product_group'].map(location_map).fillna('not register yet')  # Gunakan .loc[] di sini

    # Select the relevant columns
    #result_product = dedup_data_retail_cleanup[['product_id', 'product_group', 'location']]
    result_product = dedup_data_retail_cleanup[['product_group', 'location']].copy()
    result_product.loc[:, 'productID'] = range(1, len(result_product)+1)
    
    
    return result_product

In [52]:
hasil = product_table(data_retail_cleanup)

hasil

,product_group,location,productID
65,kitchenware,Lt.1_Stand.4_kitchenware,1
20,accessories,Lt.2_Stand.1_accessories,2
40,bag,Lt.2_Stand.4_bag,3
17,utensil,Lt.1_Stand.7_utensil,4
5,accessories,Lt.2_Stand.1_accessories,5
7,stationery,Lt.2_Stand.2_stationery,6
43,accessories,Lt.2_Stand.1_accessories,7


In [44]:
#table order
def order_table()

      name  sequential_number
0    Alice                  1
1      Bob                  2
2  Charlie                  3
3    David                  4
